<a href="https://colab.research.google.com/github/AndresNamm/CostFunctions/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


# Logistic function

# t
# Dataset
data = {
    'Age': [22, 25, 47, 52, 46, 56, 26, 27, 48, 50],
    'Salary': [50000, 60000, 150000, 200000, 90000, 160000, 80000, 58000, 140000, 135000],
    'Purchased': [0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
}

df = pd.DataFrame(data)

df.dtypes

Age          int64
Salary       int64
Purchased    int64
dtype: object

[More on sigmoid and general logistic functions](https://github.com/AndresNamm/CostFunctions/blob/main/ClassPredictionFunctions.ipynb)

In [ ]:

# Parameters
beta_0 = 0.0
beta_1 = 0.01
beta_2 = 0.0001


# Compute predicted probabilities
df['LinearComb'] = float(beta_0) + beta_1 * df['Age'] + beta_2 * df['Salary'].astype(float)
df['PredictedProb'] = sigmoid(df['LinearComb'])

# Compute log-likelihood
df['LogLikelihood'] = df['Purchased'].astype(float) * np.log(df['PredictedProb']) + (1 - df['Purchased'].astype(float)) * np.log(1 - df['PredictedProb'])
log_likelihood = df['LogLikelihood'].sum()

log_likelihood



np.float64(-25.8099679936351)

[Reference How Negative Binary Log Likelihood Cost function behaves with different examples](https://github.com/AndresNamm/CostFunctions/blob/main/BinaryNegativeLikelihoodVisualization.ipynb)

In [ ]:
df

,Age,Salary,Purchased,LinearComb,PredictedProb,LogLikelihood
0,22,50000,0,5.22,0.994622,-5.225393e+00
1,25,60000,0,6.25,0.998073,-6.251929e+00
2,47,150000,1,15.47,1.000000,-1.911896e-07
3,52,200000,1,20.52,1.000000,-1.225398e-09
4,46,90000,1,9.46,0.999922,-7.790356e-05
5,56,160000,1,16.56,1.000000,-6.428111e-08
6,26,80000,0,8.26,0.999741,-8.260259e+00
7,27,58000,0,6.07,0.997694,-6.072309e+00
8,48,140000,1,14.48,0.999999,-5.145360e-07
9,50,135000,1,14.00,0.999999,-8.315284e-07


In [ ]:
# prompt: Now generate optimization code for the previous cell

import numpy as np


learning_rate = 0.001
num_iterations = 1000

# Initialize coefficients
beta_0 = 0
beta_1 = 0.01
beta_2 = 0.0001

# Gradient descent loop
for i in range(num_iterations):
    # Compute predicted probabilities
    df['LinearComb'] = beta_0 + beta_1 * df['Age'] + beta_2 * df['Salary']
    df['PredictedProb'] = sigmoid(df['LinearComb'])

    # Compute gradients
    gradient_beta_0 = -2 * df['Purchased'].sum() + 2 * (df['PredictedProb'] * (1 - df['Purchased'])).sum()
    gradient_beta_1 = -2 * df['Age'].dot(df['Purchased'] - df['PredictedProb'])
    gradient_beta_2 = -2 * df['Salary'].dot(df['Purchased'] - df['PredictedProb'])

    # Update coefficients
    beta_0 -= learning_rate * gradient_beta_0
    beta_1 -= learning_rate * gradient_beta_1
    beta_2 -= learning_rate * gradient_beta_2

# Compute final log-likelihood
df['LogLikelihood'] = df['Purchased'] * np.log(df['PredictedProb']) + (1 - df['Purchased']) * np.log(1 - df['PredictedProb'])
log_likelihood = df['LogLikelihood'].sum()

print(f"Final log-likelihood: {log_likelihood}")


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs

Final log-likelihood: -inf


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
